In [3]:
from livekit.agents import (
    get_job_context,
    function_tool,
    jupyter,
    Agent,
    AgentSession,
    JobContext,
    RunContext,
    WorkerOptions,
    cli,
    function_tool,
)
from livekit.agents import AgentTask
from livekit.plugins import deepgram, elevenlabs, openai, silero,google
from dotenv import load_dotenv
from dataclasses import dataclass
_ = load_dotenv(override=True)

ImportError: cannot import name 'AgentTask' from 'livekit.agents' (C:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\livekit\agents\__init__.py)

In [ ]:
class CollectConsent(AgentTask[bool]):
    def __init__(self):
        super().__init__(
            instructions="Ask for recording consent and get a clear yes or no answer."
        )

    async def on_enter(self) -> None:
        await self.session.generate_reply(instructions="Ask for permission to record the call for quality assurance purposes.")

    @function_tool
    async def consent_given(self) -> None:
        """Use this when the user gives consent to record."""
        self.complete(True)

    @function_tool
    async def consent_denied(self) -> None:
        """Use this when the user denies consent to record."""
        self.complete(False)

In [ ]:
class CustomerServiceAgent(Agent):
    def __init__(self):
        super().__init__(instructions="You are a friendly customer service representative.")

    async def on_enter(self) -> None:
        if await CollectConsent(chat_ctx=self.chat_ctx):
            await self.session.generate_reply(instructions="Offer your assistance to the user.")
        else:
            await self.session.generate_reply(instructions="Inform the user that you are unable to proceed and will end the call.")
            job_ctx = get_job_context()
            await job_ctx.api.room.delete_room(api.DeleteRoomRequest(room=job_ctx.room.name))

In [ ]:
async def entryPoint(ctx:JobContext):
    await ctx.connect()
    # agent = Agent(instructions= "You are a vary funny but intelligent and helpful assistent.your responce always are funny,informative and with good humer.",
    # tools=[information,transfer_to_serious],)
    session = AgentSession[MySessionInfo](userdata=MySessionInfo(),vad=silero.VAD.load(),
                                            stt=deepgram.STT(model="nova-3"),
                                            llm=openai.LLM(model="gpt-4o-mini"),
                                            tts=elevenlabs.TTS(voice_id="JBFqnCBsd6RMkjVDRZzb"), )
    await session.start(agent=IntakeAgent(),room=ctx.room)
    

In [ ]:
jupyter.run_app(
    WorkerOptions(entrypoint_fnc=entryPoint), 
    jupyter_url="https://jupyter-api-livekit.vercel.app/api/join-token"
)